In [2]:
using AdaGram

In [4]:
vm, dict = load_model("../models/adagram/v1_d100.adagram_model")

In [ ]:
word = "apple"
prior_probs = expected_pi(vm, dict.word2id[word])
for ii in 1:5
    println(prior_probs[ii],"\t",nearest_neighbors(vm, dict, word, ii, 10))
    println()
end

In [25]:
@show disambiguate(vm, dict, "apple", split("new iphone was announced today"))
for ii in 1:5
    @show nearest_neighbors(vm, dict, "apple", ii, 5)
end

disambiguate(vm,dict,"apple",split("new iphone was announced today")) = [1.0,0.0,0.0,0.0,0.0]
nearest_neighbors(vm,dict,"apple",ii,5) = Tuple{AbstractString,Int64,Float32}[("macintosh",1,0.885584),("apple's",1,0.854207),("iigs",1,0.836565),("apple’s",1,0.834452),("macintoshes",1,0.832273)]
nearest_neighbors(vm,dict,"apple",ii,5) = Tuple{AbstractString,Int64,Float32}[("the",3,-Inf),("the",3,-Inf),("the",3,-Inf),("the",3,-Inf),("the",3,-Inf)]
nearest_neighbors(vm,dict,"apple",ii,5) = Tuple{AbstractString,Int64,Float32}[("the",3,-Inf),("the",3,-Inf),("the",3,-Inf),("the",3,-Inf),("the",3,-Inf)]
nearest_neighbors(vm,dict,"apple",ii,5) = Tuple{AbstractString,Int64,Float32}[("the",3,-Inf),("the",3,-Inf),("the",3,-Inf),("the",3,-Inf),("the",3,-Inf)]
nearest_neighbors(vm,dict,"apple",ii,5) = Tuple{AbstractString,Int64,Float32}[("the",3,-Inf),("the",3,-Inf),("the",3,-Inf),("the",3,-Inf),("the",3,-Inf)]


In [24]:
immutable AdagramModel
    vm::AdaGram.VectorModel
    dict::AdaGram.Dictionary
end
AdaGram.disambiguate(am::AdagramModel, word, context) = disambiguate(am.vm, am.dict, word, context) 


In [26]:
am=AdagramModel(vm,dict);

In [44]:
Adagram_lib = joinpath(Pkg.dir("AdaGram"), "lib", "superlib.so")

"/home/ubuntu/.julia/v0.5/AdaGram/lib/superlib.so"

In [ ]:
function log_skip_gram{T1 <: Integer, T2 <: Integer}(vm::VectorModel, w::T1, 
    s::T2, context_word_id::T1)
	code = view(vm.code, :, context_word_id)
	path = view(vm.path, :, context_word_id)
    wv = view(vm.In, :, s, w)
	return ccall((:skip_gram, "superlib"), Float32,
		(Ptr{Float32}, Ptr{Float32},
			Int, 
			Ptr{Int32}, Ptr{Int8}, Int),
		wv, vm.Out,
			, 
			path, code, size(vm.code, 1))
end

In [ ]:
"/home/ubuntu/.julia/v0.5/AdaGram/lib/superlib.so"

In [12]:
using WordNet
const WN_PATH="../data/corpora/WordNet-2.1/"
#WN_PATH3 = "/usr/share/nltk_data/corpora/wordnet/"
db = DB(WN_PATH)

WordNet.DB

In [33]:
d =  disambiguate(vm, dict, "apple", split("new iphone was announced today"))

5-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0
 0.0

In [38]:
function logprob_of_context(am::AdagramModel, context, input::Vector{Float32}; skip_oov=false, normalise_over_length=false)
    total_prob=zero(Float32)
	context_length = 0
    for context_word in context
        
        skip_oov && !haskey(am.dict.word2id, context_word) && continue
        context_length+=1
        context_word_id = am.dict.word2id[target_word]
        context_word_path = view(vm.path, :, context_word_id)
        context_word_code = view(vm.code, :, context_word_id)
        

        total_prob+= ccall((:skip_gram, "superlib"), Float32,
        ccall(Ptr{Float32}, Ptr{Float32},
			Int, 
			Ptr{Int32}, Ptr{Int8}, Int),
        wv, am.vm.Out,
			size(am.vm.In, 1), 
			context_word_path, context_word_code, size(am.vm.code, 1))
        total_prob+=word_prob
    end
    if normalise_over_length
		total_prob/=context_length #This is equivlent to taking the context_length-th root in nonlog domain. Which makes sense.
	end
	total_prob::N
end

LoadError: LoadError: UndefVarError: GenWordEmbedding not defined
while loading In[38], in expression starting on line 1

In [39]:
function synthesize_embedding(ee::AdagramModel,context, word_or_phrase)
    words = split(word_or_phrase, " ")
    for word in words
        word_id = ee.dict.word2id[word]
        probs = disambiguate(ee, word,context)
        vm.In[:, :, word)id]
    end
    
    logprobs = [Query.logprob_of_context(ee, context, wv; skip_oov=true, normalise_over_length=true) for wv in wvs]
    weighted_average(logprobs, wvs)
end


LoadError: LoadError: syntax: missing separator in array expression
while loading In[39], in expression starting on line 6